# Signal Processing and Machine Learning


### What is Signal?
- Signal is just a fucntion form to represent the information
- It combines physical simulation, mathematics and information

### What is signal processing ?
- It convert signals into another form
- Information exraction and interpretation 
    - Speech recognition
    - Computer vision

#### Examples:
 - Audio Recording : analong pressure wave is converted into one-dimentional array
 - Photos : Analog scene of light is samples using CCD array and stored as a two-dimentional array
 - Text : Collection of char which further signes with 16 bit number
 - Ratings : movies, books are stores using 0-5
 

Process: 

Model : Voices -> Feature Extrcation -> Pitch / MMCC -> Model Training 

Unknown Voice : Feature Extraction -> Pitch / MMCC -> Prediction using Model -> Speaker


#### In case of human body, 
- Activity signal -> featre extraction -> Model training -> Activity Inference

(Ref : http://web.stanford.edu/class/ee269/Lecture1.pdf)

- Generate the dataset
    - Data sourcces
    - Simulation
    - Labelling and Supervised learning 
- Preprocess and tranform data
    - Pre-Processing
    - Transformation
    - Feature extraction
    - Visulization by time, frquency, and time-frequency domain
- Develope Predictive Models
    - Refrence Models
    - Hardware-Acceleated training 
    - Analyze and tune hyper-parameter
    - Time series models : Visual Inspection of over time
    - Frequency domain analysis: Time series domain shows signal's energy distributed over time whereas, frequency domain representation includes information on the phase shift that must be applied to each frequency component in order to recover the original time signal with a combination of all the individual frequency components (FFT o DFT)
    - **LSTM for human Activity recognition (HAR)**
        - sensor based HAR
            - wearables for humanbody
        - External Device
        - Sensory output data -> input data segmentation -> LSTM based model -> combine with per time step prediction 
        - Ref : https://www.mdpi.com/1424-8220/17/11/2556/htm
             - They have used three models, 
                 - Unidirectional LSTM
                 - Bidirectional LSTM
                 - Cascade Bidorectional LSTM
- Accelerate and Deploy
    - Apps
    - Enterprise Systems
    - Embedded Devices and Hardware
    
     

For the sake of example, Human Activity dataset is used and LSTM model is applied to predict human activity.


About this Dataset

The Human Activity Recognition database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed.

In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf


In [25]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [26]:
train.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity'],
      dtype='object', length=563)

In [32]:
train.head()


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [31]:
train.shape

(7352, 563)

In [34]:
test.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,STANDING
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,2,STANDING
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,2,STANDING


In [33]:
test.shape

(2947, 563)

In [28]:
X_train = train.iloc[:,:-2]
Y_train = train.iloc[:,-1]
print(X_train.shape)
print(Y_train.shape)

(7352, 561)
(7352,)


In [30]:
X_test = test.iloc[:,:-2]
Y_test = test.iloc[:,-1]
print(X_test.shape)
print(Y_test.shape)

(2947, 561)
(2947,)


In [36]:
TRAIN_LEN = len(X_train)  
TEST_LEN = len(X_test)  
NUM_STEPS = X_train.shape[0] 
NUM_INPUT = X_train.shape[1]
NUM_HIDDEN = 32
NUM_CLASSES = 6
LR = 0.0025
LAMBDA = 0.0020
NUM_ITERS = TRAIN_LEN * 100  
BATCH_SIZE = 1024
DISP_ITER = 20000  

print(X_test.shape, Y_test.shape, X_train.shape, Y_train.shape)

(2947, 561) (2947,) (7352, 561) (7352,)


In [40]:
def LSTM_RNN(_X, _weights, _biases):
    _X = tf.transpose(_X, [1, 0, 2])
    _X = tf.reshape(_X, [-1, NUM_INPUT]) 
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    _X = tf.split(_X, NUM_STEPS, 0) 
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(NUM_HIDDEN, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(NUM_HIDDEN, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    lstm_last_output = outputs[-1]
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def next_batch(_train, step, batch_size):
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s


def one_hot(y_):
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [38]:
x = tf.placeholder(tf.float32, [None, NUM_STEPS, NUM_INPUT])
y = tf.placeholder(tf.float32, [None, NUM_CLASSES])
weights = {'hidden': tf.Variable(tf.random_normal([NUM_INPUT, NUM_HIDDEN])),'out': tf.Variable(tf.random_normal([NUM_HIDDEN, NUM_CLASSES], mean=1.0))}
biases = {'hidden': tf.Variable(tf.random_normal([NUM_HIDDEN])),'out': tf.Variable(tf.random_normal([NUM_CLASSES]))}

pred = LSTM_RNN(x, weights, biases)

l2 = LAMBDA * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2
optimizer = tf.train.AdamOptimizer(learning_rate=LR).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


KeyboardInterrupt: 

In [39]:
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []


sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

step = 1
while step * BATCH_SIZE <= NUM_ITERS:
    batch_xs = next_batch(X_Train, step, BATCH_SIZE)
    batch_ys = one_hot(next_batch(y_Train, step, BATCH_SIZE))

    
    _, loss, acc = sess.run([optimizer, cost, accuracy],feed_dict={x: batch_xs, y: batch_ys})
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    
    if (step*BATCH_SIZE % DISP_ITER == 0) or (step == 1) or (step * BATCH_SIZE > NUM_ITERS):
        print("Iteration " + str(step*BATCH_SIZE) +  ":Batch Loss = " + "{:.3f}".format(loss) + ", Accuracy = {}".format(acc))
        loss, acc = sess.run([cost, accuracy], feed_dict={x: X_Test,y: one_hot(y_Test)})
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("Test: " + "Batch Loss = {}".format(loss) + ", Accuracy = {}".format(acc))
    step += 1



one_hot_predictions, accuracy, final_loss = sess.run([pred, accuracy, cost],feed_dict={x: X_Test,y: one_hot(y_Test)})

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("RESULT: " + "Batch Loss = {}".format(final_loss) + ", Accuracy = {}".format(accuracy))



Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



AttributeError: 'list' object has no attribute 'shape'